In [1]:
import random
import numpy as np

def getKey2 (item):
    return item[0]        

def calculatePayoff(i,j, generalPayoffMatrix): 
    'it is a function that has as inputs two agents playing PD and as output the payoffs'
    if i.doCooperate == True and j.doCooperate == True:
        i.turnPayoff = generalPayoffMatrix[0, 0]
        j.turnPayoff = generalPayoffMatrix[0, 0]
        i.didOtherCoop = True
        j.didOtherCoop = True
        
    if i.doCooperate == True and j.doCooperate == False:
        i.turnPayoff = generalPayoffMatrix[0, 1]
        j.turnPayoff = generalPayoffMatrix[1, 0]
        i.didOtherCoop = False
        j.didOtherCoop = True
        
    if i.doCooperate == False and j.doCooperate == True:
        i.turnPayoff = generalPayoffMatrix[1, 0]
        j.turnPayoff = generalPayoffMatrix[0, 1]
        i.didOtherCoop = True
        j.didOtherCoop = False
        
    if i.doCooperate == False and j.doCooperate == False:
        i.turnPayoff = generalPayoffMatrix[1, 1]
        j.turnPayoff = generalPayoffMatrix[1, 1]
        i.didOtherCoop = False
        j.didOtherCoop = False
        

def listsOfProbs(listOfAgents, listOfFs, listOfPs):
    'it creates two lists ordered by the first item: one of fitnesses, one of reproduction probabilities and of death\
    probabilities in this way a simple update function can be implemented'
    
    sumOfFs = 0
    i = 0
    for i in listOfAgents:
        listOfFs.append(i.fitness)
        sumOfFs += i.fitness
        
    maxF = max(listOfFs)
    minF = min(listOfFs) 
    #mettere che se sumfs è zero e/o listof fs ha max= min allora sostituire nan con zeri
    
    if maxF != minF and sumOfFs != 0:
        for i in listOfAgents:
            temp = 1 - ((i.fitness - minF)/(maxF - minF))
            listOfPs.append([i.number, i.fitness / sumOfFs, temp])
    if maxF != minF and sumOfFs == 0:
        for i in listOfAgents:
            temp = 1 - ((i.fitness - minF)/(maxF - minF))
            listOfPs.append([i.number, 1 / len(listOfAgents), temp])
    if maxF == minF:
            listOfPs.append([i.number, 1 / len(listOfAgents), 1 / len(listOfAgents)])
    
    listOfPs = sorted(listOfPs, key = getKey2)
    listOfFs = sorted(listOfFs)
  
    return listOfFs, listOfPs #qui ritorno la lista di fitness così posso solo aggiornarla per evitare di usare troppa memoria

def dieOrReprChoice(listOfAgents, listOfPs):
    'it has as parameters the list of agents and the list of their probabilities of reproduction and death,\
    as an output gives the two agents which can reproduce or die'

    u = random.uniform(0,1)
    x = 0
    y = 0
    j = 0
    m = 0
    p1 = 0
    p2 = 0
    cumulativeDistro = []
    
    for m in range(len(listOfAgents)):
        p1 += listOfPs[m][1]
        p2 += listOfPs[m][2]
        cumulativeDistro.append([p1, p2])
    
    for j in range(len(listOfAgents)):
        if cumulativeDistro[j][0] < u:
            x += 1
        if cumulativeDistro[j][1] < u:
            y += 1

    rep = listOfAgents[x]
    die = listOfAgents[y]

    return rep, die

def die(die, listOfAgents):
    'the function removes the dying agent both from the list of agents and from each agent memory.\
    It returns the correct list of agents'
    listOfAgents = [agent for agent in listOfAgents if agent.number != die.number]
    for k in listOfAgents:
        j = 0
        iKnewHim = False
        while j in range(k.partnersMemory.shape[1]): 
            if k.partnersMemory[0, j] != die.number:
                j += 1
            else: 
                iKnewHim = True
                itsIndex = j
                j = k.partnersMemory.shape[1]
        if iKnewHim == True:
            k.partnersMemory = np.delete(k.partnersMemory, itsIndex, axis = 1)
    return listOfAgents

def varMutation(D, delta, tau, chi):
    'it is the function that mutates the parameters of the descendant of the reproducting agent, starting from the \
    values of the latter it changes them following the paper. It returns the four new values in the same order'
   
    inDelta = delta
    inTau = tau
    inChi = chi
    
    p = random.uniform(0, 1)
    if p < 0.05: 
        D = (D + 1) % 2
        
    delta = delta + random.gauss(0, 0.05)
    while delta < 0 or delta > 1: 
        delta = inDelta + random.gauss(0, 0.05)
    
    tau = tau + random.gauss(0, 0.05)    
    while tau < 0 or tau > 1: 
        tau = inTau + random.gauss(0, 0.05)
        
    chi = chi + random.gauss(0, 0.05)
    while chi < 0 or chi > 1: 
        chi = inChi + random.gauss(0, 0.05)
        
    return D, delta, tau, chi
    